In [ ]:
"""
Tests for the RequestHandler class.
"""

import asyncio
import json
import pytest
from unittest.mock import Mock, MagicMock, patch, AsyncMock

import aiohttp
import requests

from your_module.request_handler import RequestHandler


# ---------------------------------------------------------------------------
# Fixtures
# ---------------------------------------------------------------------------

@pytest.fixture
def request_handler():
    """Create a RequestHandler instance for testing."""
    return RequestHandler(
        base_url="https://api.example.com",
        default_timeout=10,
        long_timeout=60,
        max_retries=3,
        retry_delay=0.1,  # Short delay for tests
        semaphore_limit=5,
        verify_ssl=False,
    )


@pytest.fixture
def mock_response():
    """Create a mock for requests.Response."""
    response = Mock(spec=requests.Response)
    response.status_code = 200
    response.text = "Response text"
    response.json.return_value = {"key": "value"}
    return response


@pytest.fixture
def mock_aiohttp_response():
    """Create a mock for aiohttp.ClientResponse."""
    response = AsyncMock(spec=aiohttp.ClientResponse)
    response.status = 200
    response.text = AsyncMock(return_value="Response text")
    response.json = AsyncMock(return_value={"key": "value"})
    response.read = AsyncMock(return_value=b'{"key": "value"}')
    response.content = AsyncMock()
    
    # Setup streaming content
    async def mock_iter_chunked(chunk_size):
        yield b'{"streaming": true, "stream_data": {"text": "Streaming text"}}'
        yield b'{"final_response": {"answer": "Final answer"}}'
    
    response.content.iter_chunked = mock_iter_chunked
    return response


@pytest.fixture
def mock_stream_response():
    """Create a mock for a streaming requests.Response."""
    response = Mock(spec=requests.Response)
    response.status_code = 200
    
    # Setup streaming content
    def mock_iter_content(chunk_size=None):
        yield b'{"streaming": true, "stream_data": {"text": "Streaming text"}}'
        yield b'{"final_response": {"answer": "Final answer"}}'
    
    response.iter_content = mock_iter_content
    return response


# ---------------------------------------------------------------------------
# Synchronous HTTP Method Tests
# ---------------------------------------------------------------------------

@patch('requests.post')
def test_post(mock_post, request_handler, mock_response):
    """Test POST method returns response on success."""
    mock_post.return_value = mock_response
    
    response = request_handler.post('/endpoint', {"Content-Type": "application/json"}, {"data": "value"})
    
    assert response == mock_response
    mock_post.assert_called_once()


@patch('requests.get')
def test_get(mock_get, request_handler, mock_response):
    """Test GET method returns response on success."""
    mock_get.return_value = mock_response
    
    response = request_handler.get('/endpoint', {"Content-Type": "application/json"})
    
    assert response == mock_response
    mock_get.assert_called_once()


@patch('requests.put')
def test_put(mock_put, request_handler, mock_response):
    """Test PUT method returns response on success."""
    mock_put.return_value = mock_response
    
    response = request_handler.put('/endpoint', {"Content-Type": "application/json"}, {"data": "value"})
    
    assert response == mock_response
    mock_put.assert_called_once()


@patch('requests.delete')
def test_delete(mock_delete, request_handler, mock_response):
    """Test DELETE method returns response on success."""
    mock_delete.return_value = mock_response
    
    response = request_handler.delete('/endpoint', {"Content-Type": "application/json"})
    
    assert response == mock_response
    mock_delete.assert_called_once()


@patch('requests.post')
def test_post_stream(mock_post, request_handler, mock_stream_response):
    """Test post_stream method returns response on success."""
    mock_post.return_value = mock_stream_response
    
    response = request_handler.post_stream('/endpoint', {"Content-Type": "application/json"}, {"data": "value"})
    
    assert response == mock_stream_response
    mock_post.assert_called_once_with(
        'https://api.example.com/endpoint',
        headers={"Content-Type": "application/json"},
        json={"data": "value"},
        stream=True,
        verify=False,
        timeout=60
    )


# ---------------------------------------------------------------------------
# Asynchronous HTTP Method Tests
# ---------------------------------------------------------------------------

@pytest.mark.asyncio
async def test_async_post(request_handler, mock_aiohttp_response):
    """Test async_post method returns response on success."""
    with patch('aiohttp.ClientSession.post', new_callable=AsyncMock) as mock_post:
        mock_post.return_value = mock_aiohttp_response
        
        response = await request_handler.async_post('/endpoint', {"Content-Type": "application/json"}, {"data": "value"})
        
        assert response == mock_aiohttp_response
        mock_post.assert_called_once()


@pytest.mark.asyncio
async def test_async_get(request_handler, mock_aiohttp_response):
    """Test async_get method returns response on success."""
    with patch('aiohttp.ClientSession.get', new_callable=AsyncMock) as mock_get:
        mock_get.return_value = mock_aiohttp_response
        
        response = await request_handler.async_get('/endpoint', {"Content-Type": "application/json"})
        
        assert response == mock_aiohttp_response
        mock_get.assert_called_once()


@pytest.mark.asyncio
async def test_async_put(request_handler, mock_aiohttp_response):
    """Test async_put method returns response on success."""
    with patch('aiohttp.ClientSession.put', new_callable=AsyncMock) as mock_put:
        mock_put.return_value = mock_aiohttp_response
        
        response = await request_handler.async_put('/endpoint', {"Content-Type": "application/json"}, {"data": "value"})
        
        assert response == mock_aiohttp_response
        mock_put.assert_called_once()


@pytest.mark.asyncio
async def test_async_delete(request_handler, mock_aiohttp_response):
    """Test async_delete method returns response on success."""
    with patch('aiohttp.ClientSession.delete', new_callable=AsyncMock) as mock_delete:
        mock_delete.return_value = mock_aiohttp_response
        
        response = await request_handler.async_delete('/endpoint', {"Content-Type": "application/json"})
        
        assert response == mock_aiohttp_response
        mock_delete.assert_called_once()


@pytest.mark.asyncio
async def test_async_post_stream(request_handler, mock_aiohttp_response):
    """Test async_post_stream method returns response on success."""
    with patch('aiohttp.ClientSession.post', new_callable=AsyncMock) as mock_post:
        mock_post.return_value = mock_aiohttp_response
        
        response = await request_handler.async_post_stream('/endpoint', {"Content-Type": "application/json"}, {"data": "value"})
        
        assert response == mock_aiohttp_response
        mock_post.assert_called_once()


# ---------------------------------------------------------------------------
# Response Processing Tests
# ---------------------------------------------------------------------------

def test_process_response_json(request_handler, mock_response):
    """Test process_response handles JSON responses."""
    result = request_handler.process_response(mock_response)
    
    assert result == {"key": "value"}
    mock_response.json.assert_called_once()


def test_process_response_text(request_handler, mock_response):
    """Test process_response falls back to text when JSON fails."""
    mock_response.json.side_effect = ValueError("Invalid JSON")
    
    result = request_handler.process_response(mock_response)
    
    assert result == "Response text"


def test_process_response_none(request_handler):
    """Test process_response handles None input."""
    result = request_handler.process_response(None)
    
    assert result is None


@pytest.mark.asyncio
async def test_process_async_response_json(request_handler, mock_aiohttp_response):
    """Test process_async_response handles JSON responses."""
    result = await request_handler.process_async_response(mock_aiohttp_response)
    
    assert result == {"key": "value"}
    mock_aiohttp_response.json.assert_called_once()


@pytest.mark.asyncio
async def test_process_async_response_text(request_handler, mock_aiohttp_response):
    """Test process_async_response falls back to text when JSON fails."""
    mock_aiohttp_response.json.side_effect = ValueError("Invalid JSON")
    
    result = await request_handler.process_async_response(mock_aiohttp_response)
    
    assert result == "Response text"
    mock_aiohttp_response.text.assert_called_once()


@pytest.mark.asyncio
async def test_process_async_response_none(request_handler):
    """Test process_async_response handles None input."""
    result = await request_handler.process_async_response(None)
    
    assert result is None


# ---------------------------------------------------------------------------
# Streaming Tests
# ---------------------------------------------------------------------------

def test_process_stream_response(request_handler, mock_stream_response):
    """Test process_stream_response extracts final answer."""
    with patch('builtins.print') as mock_print:
        result = request_handler.process_stream_response(mock_stream_response)
        
        assert result == "Final answer"
        mock_print.assert_called_with("Streaming text", end="")


@pytest.mark.asyncio
async def test_process_async_stream_response(request_handler, mock_aiohttp_response):
    """Test process_async_stream_response extracts final answer."""
    with patch('builtins.print') as mock_print:
        result = await request_handler.process_async_stream_response(mock_aiohttp_response)
        
        assert result == "Final answer"
        mock_print.assert_called_with("Streaming text", end="")


@pytest.mark.asyncio
async def test_simple_async_stream(request_handler, mock_aiohttp_response):
    """Test simple_async_stream combines post and process."""
    with patch.object(request_handler, 'async_post_stream', new_callable=AsyncMock) as mock_post_stream:
        with patch.object(request_handler, 'process_async_stream_response', new_callable=AsyncMock) as mock_process:
            mock_post_stream.return_value = mock_aiohttp_response
            mock_process.return_value = "Final answer"
            
            result = await request_handler.simple_async_stream('/endpoint', {}, {})
            
            assert result == "Final answer"
            mock_post_stream.assert_called_once()
            mock_process.assert_called_with(mock_aiohttp_response)


@pytest.mark.asyncio
async def test_stream_chunks(request_handler, mock_aiohttp_response):
    """Test stream_chunks yields JSON objects."""
    callback_results = []
    
    def callback(text):
        callback_results.append(text)
    
    results = []
    async for json_data in request_handler.stream_chunks(mock_aiohttp_response, callback):
        results.append(json_data)
    
    assert len(results) == 2
    assert results[0] == {"streaming": True, "stream_data": {"text": "Streaming text"}}
    assert results[1] == {"final_response": {"answer": "Final answer"}}
    assert callback_results == ["Streaming text"]


# ---------------------------------------------------------------------------
# Retry Logic Tests
# ---------------------------------------------------------------------------

@patch('requests.post')
def test_post_with_retry_success(mock_post, request_handler, mock_response):
    """Test _post_with_retry succeeds on first try."""
    mock_post.return_value = mock_response
    
    response = request_handler._post_with_retry('/endpoint', {}, {})
    
    assert response == mock_response
    assert mock_post.call_count == 1


@patch('requests.post')
def test_post_with_retry_fails_then_succeeds(mock_post, request_handler, mock_response):
    """Test _post_with_retry retries after failure."""
    # First call raises exception, second call succeeds
    mock_post.side_effect = [Exception("Connection error"), mock_response]
    
    response = request_handler._post_with_retry('/endpoint', {}, {})
    
    assert response == mock_response
    assert mock_post.call_count == 2


@patch('requests.post')
def test_post_with_retry_all_fails(mock_post, request_handler):
    """Test _post_with_retry returns None after all retries fail."""
    # All calls raise exception
    mock_post.side_effect = Exception("Connection error")
    
    response = request_handler._post_with_retry('/endpoint', {}, {}, retries=3)
    
    assert response is None
    assert mock_post.call_count == 3


@patch('requests.post')
def test_post_with_retry_4xx_no_retry(mock_post, request_handler, mock_response):
    """Test _post_with_retry doesn't retry on 4xx errors."""
    mock_response.status_code = 404
    mock_post.return_value = mock_response
    
    response = request_handler._post_with_retry('/endpoint', {}, {})
    
    assert response == mock_response
    assert mock_post.call_count == 1


@patch('requests.post')
def test_post_with_retry_5xx_retry(mock_post, request_handler, mock_response):
    """Test _post_with_retry retries on 5xx errors."""
    error_response = Mock(spec=requests.Response)
    error_response.status_code = 500
    error_response.text = "Server error"
    
    # First call returns 5xx, second call succeeds
    mock_post.side_effect = [error_response, mock_response]
    
    response = request_handler._post_with_retry('/endpoint', {}, {})
    
    assert response == mock_response
    assert mock_post.call_count == 2


@pytest.mark.asyncio
async def test_async_post_with_retry_success(request_handler, mock_aiohttp_response):
    """Test _async_post_with_retry succeeds on first try."""
    with patch('aiohttp.ClientSession.post', new_callable=AsyncMock) as mock_post:
        mock_post.return_value = mock_aiohttp_response
        
        response = await request_handler._async_post_with_retry('/endpoint', {}, {})
        
        assert response == mock_aiohttp_response
        assert mock_post.call_count == 1


@pytest.mark.asyncio
async def test_async_post_with_retry_fails_then_succeeds(request_handler, mock_aiohttp_response):
    """Test _async_post_with_retry retries after failure."""
    with patch('aiohttp.ClientSession.post', new_callable=AsyncMock) as mock_post:
        # First call raises exception, second call succeeds
        mock_post.side_effect = [Exception("Connection error"), mock_aiohttp_response]
        
        response = await request_handler._async_post_with_retry('/endpoint', {}, {})
        
        assert response == mock_aiohttp_response
        assert mock_post.call_count == 2


# ---------------------------------------------------------------------------
# Error Handling Tests
# ---------------------------------------------------------------------------

@patch('requests.post')
def test_post_exception(mock_post, request_handler):
    """Test post handles exceptions."""
    mock_post.side_effect = Exception("Test exception")
    
    response = request_handler.post('/endpoint', {}, {})
    
    assert response is None


@patch('requests.get')
def test_get_exception(mock_get, request_handler):
    """Test get handles exceptions."""
    mock_get.side_effect = Exception("Test exception")
    
    response = request_handler.get('/endpoint', {})
    
    assert response is None


@pytest.mark.asyncio
async def test_async_post_exception(request_handler):
    """Test async_post handles exceptions."""
    with patch('aiohttp.ClientSession.post', new_callable=AsyncMock) as mock_post:
        mock_post.side_effect = Exception("Test exception")
        
        response = await request_handler.async_post('/endpoint', {}, {})
        
        assert response is None


@pytest.mark.asyncio
async def test_async_get_exception(request_handler):
    """Test async_get handles exceptions."""
    with patch('aiohttp.ClientSession.get', new_callable=AsyncMock) as mock_get:
        mock_get.side_effect = Exception("Test exception")
        
        response = await request_handler.async_get('/endpoint', {})
        
        assert response is None


@pytest.mark.asyncio
async def test_async_post_stream_exception(request_handler):
    """Test async_post_stream handles exceptions."""
    with patch('aiohttp.ClientSession.post', new_callable=AsyncMock) as mock_post:
        mock_post.side_effect = Exception("Test exception")
        
        response = await request_handler.async_post_stream('/endpoint', {}, {})
        
        assert response is None


def test_process_stream_response_exception(request_handler, mock_stream_response):
    """Test process_stream_response handles exceptions."""
    mock_stream_response.iter_content.side_effect = Exception("Test exception")
    
    result = request_handler.process_stream_response(mock_stream_response)
    
    assert result is None


@pytest.mark.asyncio
async def test_process_async_stream_response_exception(request_handler, mock_aiohttp_response):
    """Test process_async_stream_response handles exceptions."""
    mock_aiohttp_response.content.iter_chunked.side_effect = Exception("Test exception")
    
    result = await request_handler.process_async_stream_response(mock_aiohttp_response)
    
    assert result is None


@pytest.mark.asyncio
async def test_stream_chunks_exception(request_handler, mock_aiohttp_response):
    """Test stream_chunks handles exceptions."""
    mock_aiohttp_response.content.iter_chunked.side_effect = Exception("Test exception")
    
    results = []
    async for json_data in request_handler.stream_chunks(mock_aiohttp_response):
        results.append(json_data)
    
    assert len(results) == 0


if __name__ == "__main__":
